In [36]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import nltk
import numpy as np
import requests
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.corpus import treebank, brown, conll2000
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Part-of-Speech Tagging with a Bidirectional LSTM

In [2]:
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


True

In [3]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [4]:
# Download all PoS-tagged sentences and place them in one list.
tagged_sentences = treebank.tagged_sents(tagset='universal') +\
                   brown.tagged_sents(tagset='universal') +\
                   conll2000.tagged_sents(tagset='universal')

print(tagged_sentences[0])
print(f"Dataset size: {len(tagged_sentences)}")

[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]
Dataset size: 72202


In [5]:
sentences, sentence_tags = [], []

for s in tagged_sentences:
  sentence, tags = zip(*s)
  sentences.append(list(sentence))
  sentence_tags.append(list(tags))

The sentences and their respective tags are now in separate lists.

In [6]:
print(sentences[0])
print(sentence_tags[0])

['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


In [7]:
print(len(sentences), len(sentence_tags))

72202 72202


In [8]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

x_train, x_test, y_train, y_test = train_test_split(sentences, sentence_tags,
                                                    test_size=1 - train_ratio,
                                                    random_state=1)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test,
                                                test_size=test_ratio/(test_ratio + validation_ratio),
                                                random_state=1)

In [9]:
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))
print(len(x_test), len(y_test))

54151 54151
10830 10830
7221 7221


In [10]:
sentence_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')

In [11]:
sentence_tokenizer.fit_on_texts(x_train)

In [12]:
print(f"Vocabulary size: {len(sentence_tokenizer.word_index)}")

Vocabulary size: 52041


In [13]:
tag_tokenizer = keras.preprocessing.text.Tokenizer()
tag_tokenizer.fit_on_texts(y_train)

In [14]:
print(f"Number of PoS tags: {len(tag_tokenizer.word_index)}\n")
tag_tokenizer.get_config()

Number of PoS tags: 12



{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': False,
 'oov_token': None,
 'document_count': 54151,
 'word_counts': '{"det": 126968, "verb": 174593, "adj": 80523, "adp": 136453, "noun": 286676, "adv": 51205, ".": 142935, "pron": 44684, "conj": 35060, "num": 21461, "prt": 31229, "x": 6090}',
 'word_docs': '{".": 53332, "noun": 51171, "adv": 29531, "det": 44747, "verb": 50837, "adj": 36344, "adp": 43855, "conj": 24383, "num": 11964, "pron": 26965, "prt": 21777, "x": 2682}',
 'index_docs': '{"3": 53332, "1": 51171, "7": 29531, "5": 44747, "2": 50837, "6": 36344, "4": 43855, "9": 24383, "11": 11964, "8": 26965, "10": 21777, "12": 2682}',
 'index_word': '{"1": "noun", "2": "verb", "3": ".", "4": "adp", "5": "det", "6": "adj", "7": "adv", "8": "pron", "9": "conj", "10": "prt", "11": "num", "12": "x"}',
 'word_index': '{"noun": 1, "verb": 2, ".": 3, "adp": 4, "det": 5, "adj": 6, "adv": 7, "pron": 8, "conj": 9, "prt": 10, "

In [15]:
# The set of universal PoS tags.
tag_tokenizer.word_index

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

In [16]:
x_train_seqs = sentence_tokenizer.texts_to_sequences(x_train)

In [17]:
print(x_train_seqs[0])

[27, 86, 21, 479, 7, 2, 920, 10903, 20547, 3327, 5644, 337, 4]


In [18]:
print(f"Original: {x_train[0]}")
print(f"Reconstructed: {sentence_tokenizer.sequences_to_texts([x_train_seqs[0]])}")

Original: ['This', 'may', 'be', 'due', 'to', 'the', 'heavy', 'interlobular', 'connective', 'tissue', 'barriers', 'present', '.']
Reconstructed: ['this may be due to the heavy interlobular connective tissue barriers present .']


Next, we'll vectorize the labels (i.e. sequences of PoS tags) using its respective tokenizer.

In [19]:
y_train_seqs = tag_tokenizer.texts_to_sequences(y_train)

In [20]:
tag_tokenizer.sequences_to_texts([y_train_seqs[0]])

['det verb verb adj adp det adj adj adj noun noun adv .']

Finally, we'll do the same with the validation inputs and labels.

In [21]:
x_val_seqs = sentence_tokenizer.texts_to_sequences(x_val)
y_val_seqs = tag_tokenizer.texts_to_sequences(y_val)

In [22]:
MAX_LENGTH = len(max(x_train_seqs, key=len))
print(f"Length of longest input sequence: {MAX_LENGTH}")

Length of longest input sequence: 161


In [23]:
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train_seqs, padding='post',
                                                            maxlen=MAX_LENGTH)

In [24]:
print(x_train_padded[0])

[   27    86    21   479     7     2   920 10903 20547  3327  5644   337
     4     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


We'll do the same with the training label (PoS sequences)...

In [25]:
y_train_padded = keras.preprocessing.sequence.pad_sequences(y_train_seqs, padding='post',
                                                            maxlen=MAX_LENGTH)

...and the validation dataset.

In [26]:
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val_seqs, padding='post', maxlen=MAX_LENGTH)
y_val_padded = keras.preprocessing.sequence.pad_sequences(y_val_seqs, padding='post', maxlen=MAX_LENGTH)

In [27]:
y_train_categoricals = keras.utils.to_categorical(y_train_padded)

The label (PoS tag sequence) for a single sentence is now a **sequence of one-hot encodings**. These will serve as our training targets.

In [28]:
# The one hot encodings for the first label.
print(y_train_categoricals[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [29]:
# One-hot encoding for a single tag.
print(y_train_categoricals[0][0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


We can determine the PoS tag from a one-hot encoding by seeing which index is set to 1, then using that to query the tag tokenizer's *index_word* dictionary.

In [30]:
idx = np.argmax(y_train_categoricals[0][0])
print(f"Index: {idx}")

print(f"Tag: {tag_tokenizer.index_word[idx]}")

Index: 5
Tag: det


We'll one-hot encode the validation labels as well.

In [31]:
y_val_categoricals = keras.utils.to_categorical(y_val_padded)

In [32]:
# For the embedding layer. "+ 1" to account for the padding token.
num_tokens = len(sentence_tokenizer.word_index) + 1
embedding_dim = 128

# For the output layer. The number of classes corresponds to the
# number of possible tags.
num_classes = len(tag_tokenizer.word_index) + 1

In [33]:
# The set_seed call and kernel_initializer parameters are used here to
# ensure you and I get the same results. To get random weight initializations,
# remove them.
tf.random.set_seed(0)

model = keras.Sequential()

model.add(layers.Embedding(input_dim=num_tokens,
                           output_dim=embedding_dim,
                           input_length=MAX_LENGTH,
                           mask_zero=True))

model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True,
                                           kernel_initializer=tf.keras.initializers.random_normal(seed=1))))

model.add(layers.Dense(num_classes, activation='softmax',
                       kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


A few notes about the model summary:<br>

The embedding layer **output** has three dimensions:
- Batch size (it's showing as "None" because we didn't specify it upfront. We'll do it when we call *model.fit*).
- Sequence length (the sequences are all the same length now after our padding step).
- Embedding dimension.
<br><br>

The LSTM outputs a vector *twice* the size of what we specified because it's bidirectional. 
<br><br>

The final layer's **output** also has three dimensions:
- Batch size
- Sequence length
- Output dimension (the number of possible tags).

The output will be a **sequence of probability distributions** for each input sequence. One probability distribution per tag.



In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 161, 128)          6661376   
                                                                 
 bidirectional (Bidirection  (None, 161, 256)          263168    
 al)                                                             
                                                                 
 dense (Dense)               (None, 161, 13)           3341      
                                                                 
Total params: 6927885 (26.43 MB)
Trainable params: 6927885 (26.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train_padded, y_train_categoricals, epochs=20,
                    batch_size=256, validation_data=(x_val_padded, y_val_categoricals),
                    callbacks=[es_callback])

Epoch 1/20
212/212 [==============================] - 36s 118ms/step - loss: 0.9803 - accuracy: 0.7025 - val_loss: 0.1541 - val_accuracy: 0.9524
Epoch 2/20
212/212 [==============================] - 17s 82ms/step - loss: 0.1105 - accuracy: 0.9647 - val_loss: 0.1040 - val_accuracy: 0.9651
Epoch 3/20
212/212 [==============================] - 10s 48ms/step - loss: 0.0727 - accuracy: 0.9761 - val_loss: 0.0949 - val_accuracy: 0.9678
Epoch 4/20
212/212 [==============================] - 8s 39ms/step - loss: 0.0571 - accuracy: 0.9812 - val_loss: 0.0925 - val_accuracy: 0.9691
Epoch 5/20
212/212 [==============================] - 9s 40ms/step - loss: 0.0462 - accuracy: 0.9850 - val_loss: 0.0909 - val_accuracy: 0.9706
Epoch 6/20
212/212 [==============================] - 8s 36ms/step - loss: 0.0373 - accuracy: 0.9881 - val_loss: 0.0935 - val_accuracy: 0.9706
Epoch 7/20
212/212 [==============================] - 7s 33ms/step - loss: 0.0303 - accuracy: 0.9906 - val_loss: 0.0969 - val_accuracy: 0.

Once our model is trained, we'll vectorize and pad the testing dataset. In the case of the labels, we'll also one-hot encode them.

In [37]:
# Save the model
model.save('/content/drive/MyDrive/RNN model/bidirectional_lstm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
# Preprocess the test data and test the model.
x_test_seqs = sentence_tokenizer.texts_to_sequences(x_test)
x_test_padded = keras.preprocessing.sequence.pad_sequences(x_test_seqs, padding='post', maxlen=MAX_LENGTH)

y_test_seqs = tag_tokenizer.texts_to_sequences(y_test)
y_test_padded = keras.preprocessing.sequence.pad_sequences(y_test_seqs, padding='post', maxlen=MAX_LENGTH)
y_test_categoricals = keras.utils.to_categorical(y_test_padded)

In [39]:
model.evaluate(x_test_padded, y_test_categoricals)

226/226 [==============================] - 2s 7ms/step - loss: 0.1092 - accuracy: 0.9694


[0.10918478667736053, 0.969357967376709]

We can now use our model to tag sentences.

In [40]:
samples = [
    "Brown refused to testify.",
    "Brown sofas are on sale.",
]

The function below takes a list of strings, tokenizes and pads them, then has the model tag them. Note that if a sentence is longer than MAX_LENGTH, it'll be truncated.

In [41]:
def tag_sentences(sentences):
  sentences_seqs = sentence_tokenizer.texts_to_sequences(sentences)
  sentences_padded = keras.preprocessing.sequence.pad_sequences(sentences_seqs,
                                                                maxlen=MAX_LENGTH,
                                                                padding='post')

  # The model returns a LIST of PROBABILITY DISTRIBUTIONS (due to the softmax)
  # for EACH sentence. There is one probability distribution for each PoS tag.
  tag_preds = model.predict(sentences_padded)

  sentence_tags = []

  # For EACH LIST of probability distributions...
  for i, preds in enumerate(tag_preds):

    # Extract the most probable tag from EACH probability distribution.
    # Note how we're extracting tags for only the non-padding tokens.
    tags_seq = [np.argmax(p) for p in preds[:len(sentences_seqs[i])]]

    # Convert the sentence and tag sequences back to their token counterparts.
    words = [sentence_tokenizer.index_word[w] for w in sentences_seqs[i]]
    tags = [tag_tokenizer.index_word[t] for t in tags_seq]
    sentence_tags.append(list(zip(words, tags)))

  return sentence_tags


In [42]:
tagged_sample_sentences = tag_sentences(samples)

1/1 [==============================] - 2s 2s/step


In [43]:
print(tagged_sample_sentences[0])

[('brown', 'noun'), ('refused', 'verb'), ('to', 'prt'), ('testify', 'verb')]


In [44]:
print(tagged_sample_sentences[1])

[('brown', 'adj'), ('sofas', 'noun'), ('are', 'verb'), ('on', 'adp'), ('sale', 'noun')]


# Language Modelling With Stacked LSTMs

We'll build a language model trained on the *Art of War* by Sun Tzu.

In [45]:
# Open the file in read mode
with open('/content/drive/MyDrive/RNN model/art_of_war.txt', 'r') as file:
    # Read the entire content of the file
    art_of_war = file.read()
    print(art_of_war)


1. Sun Tzŭ said: The art of war is of vital importance to the State.

2. It is a matter of life and death, a road either to safety or to
ruin. Hence it is a subject of inquiry which can on no account be
neglected.

3. The art of war, then, is governed by five constant factors, to be
taken into account in one’s deliberations, when seeking to determine
the conditions obtaining in the field.

4. These are: (1) The Moral Law; (2) Heaven; (3) Earth; (4) The
Commander; (5) Method and discipline.

5, 6. _The Moral Law_ causes the people to be in complete accord with
their ruler, so that they will follow him regardless of their lives,
undismayed by any danger.

7. Heaven signifies night and day, cold and heat, times and seasons.

8. Earth comprises distances, great and small; danger and security;
open ground and narrow passes; the chances of life and death.

9. The Commander stands for the virtues of wisdom, sincerity,
benevolence, courage and strictness.

10. By _Method and discipline_ are to

In [46]:
art_of_war[:300]

'1. Sun Tzŭ said: The art of war is of vital importance to the State.\n\n2. It is a matter of life and death, a road either to safety or to\nruin. Hence it is a subject of inquiry which can on no account be\nneglected.\n\n3. The art of war, then, is governed by five constant factors, to be\ntaken into accou'

In [48]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [49]:
tokenizer.fit_on_texts([art_of_war])

The tokenizer's internal dictionary now maps characters rather than words...

In [50]:
tokenizer.get_config()

{'num_words': None,
 'filters': '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
 'lower': True,
 'split': ' ',
 'char_level': True,
 'oov_token': None,
 'document_count': 1,
 'word_counts': '{"1": 179, ".": 896, " ": 9794, "s": 3081, "u": 1467, "n": 3565, "t": 4398, "z": 20, "\\u016d": 13, "a": 3475, "i": 3573, "d": 1681, ":": 48, "h": 2558, "e": 5837, "r": 2776, "o": 3548, "f": 1238, "w": 981, "v": 478, "l": 1722, "m": 1201, "p": 769, "c": 1390, "\\n": 1443, "2": 127, ",": 634, "y": 1055, "b": 708, "j": 23, "q": 55, "g": 1007, "3": 87, "k": 345, "\\u2019": 57, "4": 66, "(": 59, ")": 59, ";": 168, "5": 58, "6": 51, "_": 62, "7": 39, "8": 36, "9": 34, "0": 38, "x": 49, "\\u2014": 16, "?": 8, "!": 8, "-": 57, "\\u201c": 3, "\\u201d": 3, "\\u0153": 7, "\\u00fc": 3, "\\u2018": 1}',
 'word_docs': '{"e": 1, "m": 1, "\\u00fc": 1, "\\u201d": 1, "o": 1, "_": 1, " ": 1, "q": 1, "0": 1, ")": 1, "f": 1, ":": 1, "y": 1, "9": 1, "z": 1, "c": 1, "t": 1, "(": 1, "x": 1, "?": 1, "5": 1, "\\u2014": 1, "\\u0153"

...and the resulting possibility space is much smaller.

In [51]:
print(f"Tokenizer \"Vocabulary\" size: {len(tokenizer.word_index)}")

Tokenizer "Vocabulary" size: 56


As we did when building the PoS tagger, we'll vectorize the book's characters into a sequence of integers, each integer mapping to a particular character.

In [52]:
seq = tokenizer.texts_to_sequences([art_of_war])[0]

In [53]:
print(f"Text length: {len(seq)}")

Text length: 61054


In [54]:
# Sanity check.
tokenizer.sequences_to_texts([seq[:10]])

['1 .   s u n   t z ŭ']

In [55]:
slices = tf.data.Dataset.from_tensor_slices(seq)
type(slices)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [56]:
# Create a Dataset from the first ten slices and convert to a list to output to console.
list(slices.take(10))

[<tf.Tensor: shape=(), dtype=int32, numpy=27>,
 <tf.Tensor: shape=(), dtype=int32, numpy=21>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=8>,
 <tf.Tensor: shape=(), dtype=int32, numpy=13>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=47>,
 <tf.Tensor: shape=(), dtype=int32, numpy=49>]

In [57]:
# The first ten elements from the original vectorized corpus.
seq[:10]

[27, 21, 1, 8, 13, 5, 1, 3, 47, 49]

Here, we're creating windows of `input_timesteps + 1`. The *input_timesteps* represents our training example length. The *+1* is there to help us create the target/label for each training example. This will be clarified further below.<br><br>
In addition, we're setting *shift* to 1. This means we'll get overlapping windows shifted by 1. e.g. if the input is [1, 2, 3, 4, ...]. The first window will contain [1, 2, 3, ...], the second window will contain [2, 3, 4, ...] and so on. This is so we can have more training examples.<br><br>
Finally, we're setting *drop_remainder* to True which ensures ALL windows contain exactly N elements. i.e. once the input contains fewer than N elements, they are ignored.

In [58]:
input_timesteps = 100
window_size = input_timesteps + 1
windows = slices.window(window_size, shift=1, drop_remainder=True)

Looking at the first few windows, we can see they're all the same length and that each subsequent window is shifted over by 1. Our corpus has now been divided into segments of length `input_timesteps + 1`.

In [59]:
for w in windows.take(3):
  arr = list(w.as_numpy_iterator())
  print(len(arr), arr)

101 [27, 21, 1, 8, 13, 5, 1, 3, 47, 49, 1, 8, 7, 4, 12, 41, 1, 3, 10, 2, 1, 7, 9, 3, 1, 6, 16, 1, 20, 7, 9, 1, 4, 8, 1, 6, 16, 1, 25, 4, 3, 7, 11, 1, 4, 17, 22, 6, 9, 3, 7, 5, 15, 2, 1, 3, 6, 1, 3, 10, 2, 1, 8, 3, 7, 3, 2, 21, 14, 14, 29, 21, 1, 4, 3, 1, 4, 8, 1, 7, 1, 17, 7, 3, 3, 2, 9, 1, 6, 16, 1, 11, 4, 16, 2, 1, 7, 5, 12, 1, 12]
101 [21, 1, 8, 13, 5, 1, 3, 47, 49, 1, 8, 7, 4, 12, 41, 1, 3, 10, 2, 1, 7, 9, 3, 1, 6, 16, 1, 20, 7, 9, 1, 4, 8, 1, 6, 16, 1, 25, 4, 3, 7, 11, 1, 4, 17, 22, 6, 9, 3, 7, 5, 15, 2, 1, 3, 6, 1, 3, 10, 2, 1, 8, 3, 7, 3, 2, 21, 14, 14, 29, 21, 1, 4, 3, 1, 4, 8, 1, 7, 1, 17, 7, 3, 3, 2, 9, 1, 6, 16, 1, 11, 4, 16, 2, 1, 7, 5, 12, 1, 12, 2]
101 [1, 8, 13, 5, 1, 3, 47, 49, 1, 8, 7, 4, 12, 41, 1, 3, 10, 2, 1, 7, 9, 3, 1, 6, 16, 1, 20, 7, 9, 1, 4, 8, 1, 6, 16, 1, 25, 4, 3, 7, 11, 1, 4, 17, 22, 6, 9, 3, 7, 5, 15, 2, 1, 3, 6, 1, 3, 10, 2, 1, 8, 3, 7, 3, 2, 21, 14, 14, 29, 21, 1, 4, 3, 1, 4, 8, 1, 7, 1, 17, 7, 3, 3, 2, 9, 1, 6, 16, 1, 11, 4, 16, 2, 1, 7, 5, 12, 1, 12, 2

The *window* method returns a nested dataset of datasets (i.e. each window is a dataset containing a tensor).

In [60]:
print(windows, '\n')

for w in windows.take(2):
  print(w)

<_WindowDataset element_spec=DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([]))> 

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>


In [61]:
dataset = windows.flat_map(lambda window: window.batch(window_size))

We now have a single dataset of tensors, where each tensor is `input_timesteps+1` long and shifted by 1.

In [62]:
for d in dataset.take(2):
  print(d)

tf.Tensor(
[27 21  1  8 13  5  1  3 47 49  1  8  7  4 12 41  1  3 10  2  1  7  9  3
  1  6 16  1 20  7  9  1  4  8  1  6 16  1 25  4  3  7 11  1  4 17 22  6
  9  3  7  5 15  2  1  3  6  1  3 10  2  1  8  3  7  3  2 21 14 14 29 21
  1  4  3  1  4  8  1  7  1 17  7  3  3  2  9  1  6 16  1 11  4 16  2  1
  7  5 12  1 12], shape=(101,), dtype=int32)
tf.Tensor(
[21  1  8 13  5  1  3 47 49  1  8  7  4 12 41  1  3 10  2  1  7  9  3  1
  6 16  1 20  7  9  1  4  8  1  6 16  1 25  4  3  7 11  1  4 17 22  6  9
  3  7  5 15  2  1  3  6  1  3 10  2  1  8  3  7  3  2 21 14 14 29 21  1
  4  3  1  4  8  1  7  1 17  7  3  3  2  9  1  6 16  1 11  4 16  2  1  7
  5 12  1 12  2], shape=(101,), dtype=int32)


In [63]:
batch_size = 32

In [64]:
# The expected number of batches should be (len(seq) - input_timesteps) / batch_size
batches = dataset.shuffle(10000).batch(batch_size)

In [65]:
for b in batches.take(2):
  print(b)

tf.Tensor(
[[10  6 13 ...  2  1  2]
 [13 19 10 ... 18  8 21]
 [ 1 12  7 ...  1 22  9]
 ...
 [21 14 14 ...  2 12  1]
 [17 23  2 ...  6 13 19]
 [ 9  2  2 ... 14 35 21]], shape=(32, 101), dtype=int32)
tf.Tensor(
[[ 7  3  3 ...  1  3 10]
 [12  1  7 ...  5 12  1]
 [ 1 22  6 ...  1  6  5]
 ...
 [ 8 15  4 ...  7  5 12]
 [24  1 13 ...  2  1  9]
 [15 10  2 ...  1  7  5]], shape=(32, 101), dtype=int32)


We can now separate each example into an input sequence(x) and a corresponding label/target sequence(y).<br><br>

**Teacher Forcing** where:<br>
1. At each timestep during training, the output is compared to a label.
2. At the next timestep, rather than feeding the model the previous output, we feed it the next character of the input sequence (i.e. what the model should've outputted).
<br><br>

This is why each sequence is of size *input_timesteps + 1*. Each sequence is now going to be separated into TWO sequences. The first sequence will be the training input and will be of length *input_timesteps* (i.e. everything but the LAST character). The second sequence will be the label/target and will consist of all the sequence elements shifted by 1 (i.e. everything but the FIRST character).<br><br>

So if a sequence is "she swam in the lake", then:
- The input will be "she swam in the lak" (drop the last character)
- The target/label will be "he swam in the lake" (drop the first character)

In [66]:
xy_batches = batches.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

Each batch now consists of a set of input sequences and a corresponding set of label/target sequences, with the labels/targets shifted by 1.

In [67]:
for b in xy_batches.take(1):
  print(b)

(<tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[22,  6,  5, ...,  8,  2, 11],
       [ 6,  1,  4, ...,  2, 11, 12],
       [ 1, 12,  2, ...,  6,  9,  1],
       ...,
       [18,  1, 20, ...,  1,  7,  5],
       [ 9, 18, 24, ...,  4,  8,  1],
       [ 2,  1, 20, ..., 23, 18,  1]], dtype=int32)>, <tf.Tensor: shape=(32, 100), dtype=int32, numpy=
array([[ 6,  5,  1, ...,  2, 11, 24],
       [ 1,  4,  8, ..., 11, 12, 14],
       [12,  2, 16, ...,  9,  1, 17],
       ...,
       [ 1, 20,  4, ...,  7,  5, 12],
       [18, 24,  1, ...,  8,  1, 22],
       [ 1, 20,  2, ..., 18,  1, 15]], dtype=int32)>)


In [68]:
# For greater clarity, this is the first input sequence from the first batch,
# and it's corresponding label/target sequence.
for b in xy_batches.take(1):
  print("x1 length: ", len(b[0][0].numpy()))
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1 length: ", len(b[1][0].numpy()))
  print("y1: ", b[1][0].numpy())

x1 length:  100
x1:  [ 3  2  1 10  4 17 21 14 22  9  2  3  2  5 12  1  3  6  1 23  2  1 20  2
  7 26 24  1  3 10  7  3  1 10  2  1 17  7 18  1 19  9  6 20  1  7  9  9
  6 19  7  5  3 21 14 14 29 30 21  1  4 16  1 10  2  1  4  8  1  3  7 26
  4  5 19  1 10  4  8  1  2  7  8  2 24  1 19  4 25  2  1 10  4 17  1  5
  6  1  9  2]


y1 length:  100
y1:  [ 2  1 10  4 17 21 14 22  9  2  3  2  5 12  1  3  6  1 23  2  1 20  2  7
 26 24  1  3 10  7  3  1 10  2  1 17  7 18  1 19  9  6 20  1  7  9  9  6
 19  7  5  3 21 14 14 29 30 21  1  4 16  1 10  2  1  4  8  1  3  7 26  4
  5 19  1 10  4  8  1  2  7  8  2 24  1 19  4 25  2  1 10  4 17  1  5  6
  1  9  2  8]


The last step before we can build our model is to one-hot encode the **inputs**. We're doing this because:
1. We're not using embeddings for the input. We can, but since this is a character model with just a few dozen possible choices, we can get away with one-hot encoding. There's also no reason to think a particular letter should be closer to another in vector space as we would want in a word-level model.

2. Since we're not using embeddings and our input is categorical, we need to one-hot encode.

Note that despite our labels ALSO being categorical, we are NOT one-hot encoding them this time. This is because we'll be using a loss function that can help us skip that step (more below).

In [69]:
num_tokens = len(tokenizer.word_index) + 1

# One-hot encode the input sequences, don't do anything with the label/target sequences.
xy_batches = xy_batches.map(lambda inputs, labels: (tf.one_hot(inputs, depth=num_tokens), labels))

In [70]:
# Each input sequence is now a sequence of one-hot encodings.
for b in xy_batches.take(1):
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1: ", b[1][0].numpy())

x1:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


y1:  [ 2  7  8  6  5  8 21 14 14 44 21  1  2  7  9  3 10  1 15  6 17 22  9  4
  8  2  8  1 12  4  8  3  7  5 15  2  8 24  1 19  9  2  7  3  1  7  5 12
  1  8 17  7 11 11 28  1 12  7  5 19  2  9  1  7  5 12  1  8  2 15 13  9
  4  3 18 28 14  6 22  2  5  1 19  9  6 13  5 12  1  7  5 12  1  5  7  9
  9  6 20  1]


At this point, we've:
- Segmented our corpus into fixed-length sequences.
- Created training and label/target sequences.
- Organized them into batches.

The last step is to add some **prefetching**. This is an optimization step. This way, while the model trains on the current batch of data, the pipeline reads and prepares the next batch.<br>

In [71]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [72]:
model = keras.models.Sequential()

model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))

model.add(layers.Dense(num_tokens, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')


In [73]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, None, 128)         95232     
                                                                 
 lstm_2 (LSTM)               (None, None, 128)         131584    
                                                                 
 dense_1 (Dense)             (None, None, 57)          7353      
                                                                 
Total params: 234169 (914.72 KB)
Trainable params: 234169 (914.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Because this model takes a few hours to train, we're using **model checkpoints** to save the weights after every epoch. This way, if something goes wrong with our system during training, we can reload the last set of weights from the checkpoint, and resume training from there.<br>
https://keras.io/api/callbacks/model_checkpoint/

In [74]:
filepath="./ArtofWarLM/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history = model.fit(xy_batches, epochs=50, callbacks=[cp_callback])

In [ ]:
model.save('/content/drive/MyDrive/RNN model/art_of_war_char_level_lm')

In [78]:
model = keras.models.load_model('/content/drive/MyDrive/RNN model/art_of_war_char_level_lm')

Now that we have a trained model, let's generate some text.<br><br>
The function below takes some seed text and uses that to generate a certain number of characters. For each character, it uses the generated text so far as the input. It's not the most efficient function but it'll work here.<br><br>
There's also a *temperature* parameter. The next character is picked from a probability distribution. By dividing the log of this distribution by *temperature*, we can influence the randomness of the output.<br><br>
When the temperature is low (< 1), the probability distribution sharpens and the model will be more strict in recreating the original text. As we raise the temperature, the distribution flattens and there's a higher chance the model picks something unexpected, resulting in greater surprise in the output. In practice, a high enough temperature will result in nonsense.

In [79]:
def generate_text(model, tokenizer, seed_text, num_chars=200, temperature=1):

  text = seed_text

  for _ in range(num_chars):

    # Take the last *input_timesteps* number of characters in the text so far
    # as input.
    input = np.array(tokenizer.texts_to_sequences([text[-input_timesteps:]]))
    input = tf.one_hot(input, num_tokens)

    # Create probability distribution for next character adjusted by temperature.
    preds = model.predict(input)[0, -1:, :] # <-- We want only the last character so we're extracting the softmax output for that.
    preds = tf.math.log(preds) / temperature

    # Sample next character and add to running text.
    next_char = tf.random.categorical(preds, num_samples=1)
    next_char = tokenizer.sequences_to_texts(next_char.numpy())[0]

    text += next_char

  return text


In [80]:
%%time
print(generate_text(model, tokenizer, "Banana peels on the battlefield can", num_chars=300, temperature=0.2))

1/1 [==============================] - 0s 69ms/step
Banana peels on the battlefield can ever beseens the applies and and abroad,
and men will drop down exhausted on the highways. as many as seven
hundred thousand families will be impeded in their labor.

2. hostile armies may face each other for years, striving for the
victory which is decided in a single day. this being so, to remain
CPU times: user 39.7 s, sys: 882 ms, total: 40.6 s
Wall time: 45.3 s


In [81]:
print(generate_text(model, tokenizer, "It's time to release the Kraken when", num_chars=300, temperature=0.5))

1/1 [==============================] - 0s 65ms/step
It's time to release the Kraken when the moon is in the constellations of the sieve, the
wall, the wing or the cross-bar; for these four are all days of rising
wind.

5. in attacking with fire, one should be prepared to meet five possible
developments:

6. (1) when fire breaks out inside the enemy’s camp, respond at once
with an attac


In [82]:
print(generate_text(model, tokenizer, "Crush your enemies, see them driven before you, and", num_chars=300,
                    temperature=1))

1/1 [==============================] - 0s 112ms/step
Crush your enemies, see them driven before you, and when the momen soldiers into positions whence there is no escape, and
they will prefer death tof the
yin dy, and though starting after him to come up.

11. if the enemy his tactics him where he seeps in a real
advantage, we may everyour at its tail, and
you will be attacked by its head; striving fo


In [83]:
print(generate_text(model, tokenizer, "What is best in life?", num_chars=300, temperature=2))

1/1 [==============================] - 0s 70ms/step
What is best in life?

15. nowing our own spies to know of wind, the spoilm by is local ivolverr
besousinveres wallsh and
swampes, but thren cto wordren is attack some intactics
luvarginy inninghoulds of the hou hae utill is
tupports
matement cleads.” it
is with your litfate and to wings, when you must rush in.

66. for
